In [1]:
!nvidia-smi

Tue Apr 11 13:03:24 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 531.41                 Driver Version: 531.41       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                      TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3060 Ti    WDDM | 00000000:05:00.0  On |                  N/A |
| 30%   41C    P8               15W / 200W|    749MiB /  8192MiB |     22%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
import torch
torch.cuda.is_available()

True

In [1]:
!pip install -U pip
!pip install accelerate==0.18.0
!pip install appdirs==1.4.4
!pip install bitsandbytes==0.37.2
!pip install datasets==2.10.1
!pip install fire==0.5.0
!pip install git+https://github.com/huggingface/peft.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install torch==2.0.0
!pip install sentencepiece==0.1.97
!pip install tensorboardX==2.6
!pip install gradio==3.23.0

  Using cached accelerate-0.18.0-py3-none-any.whl (215 kB)
  Using cached PyYAML-6.0-cp310-cp310-win_amd64.whl (151 kB)
  Using cached bitsandbytes-0.37.2-py3-none-any.whl (84.2 MB)
  Using cached datasets-2.10.1-py3-none-any.whl (469 kB)
  Using cached huggingface_hub-0.13.4-py3-none-any.whl (200 kB)
  Using cached responses-0.18.0-py3-none-any.whl (38 kB)
  Using cached fsspec-2023.4.0-py3-none-any.whl (153 kB)
  Using cached aiohttp-3.8.4-cp310-cp310-win_amd64.whl (319 kB)
  Using cached xxhash-3.2.0-cp310-cp310-win_amd64.whl (30 kB)
  Using cached pandas-2.0.0-cp310-cp310-win_amd64.whl (11.2 MB)
  Using cached tqdm-4.65.0-py3-none-any.whl (77 kB)
  Using cached dill-0.3.6-py3-none-any.whl (110 kB)
  Using cached multiprocess-0.70.14-py310-none-any.whl (134 kB)
  Using cached pyarrow-11.0.0-cp310-cp310-win_amd64.whl (20.6 MB)
  Using cached multidict-6.0.4-cp310-cp310-win_amd64.whl (28 kB)
  Using cached async_timeout-4.0.2-py3-none-any.whl (5.8 kB)
  Using cached aiosignal-1.3.1-py

  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git 'C:\Users\panta\AppData\Local\Temp\pip-req-build-w3g_9rtd'


  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git 'C:\Users\panta\AppData\Local\Temp\pip-req-build-cklyb5fi'



  Cloning https://github.com/huggingface/transformers.git to c:\users\panta\appdata\local\temp\pip-req-build-cklyb5fi
  Resolved https://github.com/huggingface/transformers.git to commit 1b1867d86b9203ee8b9d41bed5d6224c7f407a18
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for transformers: filename=transformers-4.28.0.dev0-py3-none-any.whl size=6911533 sha256=75050fa0367d4f32df125e17b3649c2f3cd31c7a7ef935cf91eb3dd1baad7d24
  Stored in directory: C:\Users\panta\AppData\Local\Temp\pip-ephem-wheel-cache-vyqh1678\wheels\e7\9c\5b\e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4

In [3]:
import torch
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

'cpu'

In [2]:
import transformers
import textwrap
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import os
import sys
from typing import List
 
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_int8_training,
)
 
import fire
import torch
from datasets import load_dataset
import pandas as pd
 
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from pylab import rcParams
 
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

c:\Users\panta\anaconda3\envs\textgen\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: Required library version not found: libsbitsandbytes_cpu.so. Maybe you need to compile it from source?
CUDA SETUP: Defaulting to libbitsandbytes_cpu.so...
argument of type 'WindowsPath' is not iterable
CUDA SETUP: Required library version not found: libsbitsandbytes_cpu.so. Maybe you need to compile it from source?
CUDA SETUP: Defaulting to libbitsandbytes_cpu.so...
argument of type 'WindowsPath' is not iterable


c:\Users\panta\anaconda3\envs\textgen\lib\site-packages\bitsandbytes\cextension.py:31: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


ModuleNotFoundError: No module named 'seaborn'

In [ ]:
BASE_MODEL = "declare-lab/flan-alpaca-xl"
 
model = AutoModelForSeq2SeqLM.from_pretrained(
    BASE_MODEL,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
 
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
 
tokenizer.pad_token_id = (
    0  # unk. we want this to be different from the eos token
)
tokenizer.padding_side = "left"